In [ ]:
import numpy as np
shape = [3,5]
data = np.random.rand(*shape)


In [ ]:
data.shape
print(data)
print(data.shape)
sh = data.shape
print(sh)
do = (0,)*len(sh)
print(do)

data2 = np.arange(20).reshape(4,5)
print(data2)

In [ ]:
from wands import RandomData



In [ ]:
data2 = RandomData([3,6])
print(data2)
print(repr(data2))
print(data2[2].size)
print(data2[1,5])
print(data2.shape())
type(data2)
type(data2.get_numpy_array())
print(data2.shape())
print(data2.size)

In [ ]:
from wands import AdiosWands

In [ ]:
params = {
    "IPAddress": "0.0.0.0",
    "Port": "12306",
    "Timeout": "5",
    "TransportMode": "reliable",
}
obj = AdiosWands(link="DeclaredIO1", engine="Dataman")
obj.set_parameters(params)

In [ ]:
print(str(obj))

In [ ]:
params2 = {
    "IPAddress": "0.0.0.1",
    "Port": "12302",
    "Timeout": "5",
    "TransportMode": "reliable",
}
obj2 = AdiosWands(link="DeclaredIO2", parameters=params2)

In [ ]:
print(str(obj2))

In [ ]:
import adios2
ad = adios2.ADIOS()
ad = None
type(ad)

In [ ]:
from wands import AdiosWands
import numpy as np
import threading
import logging
import time
import concurrent.futures

def thread_sorter(name:str):
    if name=="Sender" :
        paramssend = {
            "IPAddress": "127.0.0.1",
            "Port": "12345",
            "Timeout": "6",
            "TransportMode": "fast",
            "RendezvousReaderCount": "1",
        }

        data = np.arange(20).reshape(4,5)
        logging.info(" Thread %s: creating Adiosobject",name)
        adios_s = AdiosWands(link="Sender",parameters=paramssend)
        logging.info(" Thread %s: initiating sending",name)
        adios_s.send("IOS", "testdata", data)
        logging.info(" Thread %s: sending finished",name)

    elif name=="Receiver":
        paramsrec =  {
            "IPAddress": "127.0.0.1",
            "Port": "12345",
            "Timeout": "6",
            "TransportMode": "fast",
            "RendezvousReaderCount": "1",
        }       
        logging.info(" Thread %s: creating Adiosobject",name)
        adios_r = AdiosWands(link="Reader",parameters=paramsrec)
        logging.info(" Thread %s: initiating receiving",name)
        data_r = adios_r.receive("IOR","testdata")
        logging.info(" Thread %s: finished receiving",name)

    else:
        raise ValueError("unknown threadname only Sender or Receiver")
format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO,
                    datefmt="%H:%M:%S")
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    executor.map(thread_sorter, ["Sender","Receiver"])
# paramssend = {
#     "IPAddress": "127.0.0.1",
#     "Port": "12345",
#     "Timeout": "6",
#     "TransportMode": "fast",
#     "RendezvousReaderCount": "1",
# }
# paramsrec =  {
#     "IPAddress": "127.0.0.1",
#     "Port": "12345",
#     "Timeout": "6",
#     "TransportMode": "fast",
#     "RendezvousReaderCount": "1",
# }
# data = np.arange(20).reshape(4,5)
# adios_s = AdiosWands(link="Sender",parameters=paramssend)
# adios_r = AdiosWands(link="Reader",parameters=paramsrec)

# print
# data_r = adios_r.receive("IOR","testdata")
# print("after receive")
# adios_s.send("IOS", "testdata", data)
assert np.array_equal(data, data_r)

In [ ]:

from wands import AdiosWands
import numpy as np
from multiprocessing import Pool
import logging
import time
import concurrent.futures

def thread_sorter(name:str):
    if name=="Sender" :
        paramssend = {
            "IPAddress": "127.0.0.1",
            "Port": "12345",
            "Timeout": "6",
            "TransportMode": "fast",
            "RendezvousReaderCount": "1",
        }

        data = np.arange(20).reshape(4,5)
        print(data)
        logging.info(" Thread %s: creating Adiosobject",name)
        adios_s = AdiosWands(link="Sender",parameters=paramssend)
        logging.info(" Thread %s: initiating sending",name)
        adios_s.send("IOS", "testdata", data)
        logging.info(" Thread %s: sending finished",name)
        return data

    elif name=="Receiver":
        paramsrec =  {
            "IPAddress": "127.0.0.1",
            "Port": "12345",
            "Timeout": "6",
            "TransportMode": "fast",
            "RendezvousReaderCount": "1",
        }       
        logging.info(" Thread %s: creating Adiosobject",name)
        adios_r = AdiosWands(link="Reader",parameters=paramsrec)
        logging.info(" Thread %s: initiating receiving",name)
        data_r = adios_r.receive("IOR","testdata")
        logging.info(" Thread %s: finished receiving",name)
        print(data_r)
        return data_r

    else:
        raise ValueError("unknown threadname only Sender or Receiver")
format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO,
                    datefmt="%H:%M:%S")
with Pool(2) as executor:
    executor.map(thread_sorter, ["Sender","Receiver"])

assert np.array_equal(data, data_r)


In [ ]:

from wands import AdiosWands, RandomData
import numpy as np
from multiprocessing import Process, Pipe
import logging
import time
import concurrent.futures

def thread_send(name:str):
   # assert name == "sender_proc"
    paramssend = {
        "IPAddress": "127.0.0.1",
        "Port": "12307",
        "Timeout": "6",
        "TransportMode": "reliable",
        "RendezvousReaderCount": "1",
    }
    data = name.recv()
    #data = np.arange(20).reshape(4,5)
    print(f"data on sender side {data!s}")
    #data = np.arange(20).reshape(4,5)
    logging.info(" Thread %s: creating Adiosobject",name)
    adios_s = AdiosWands(link="Sender",parameters=paramssend)
    logging.info(" Thread %s: initiating sending",name)
    adios_s.send("IOS", "testdata", data)
    logging.info(" Thread %s: sending finished",name)

def thread_receive(name:str):
   # assert name == "receiver_proc"
    paramsrec =  {
        "IPAddress": "127.0.0.1",
        "Port": "12307",
        "Timeout": "6",
        "TransportMode": "reliable",
        "RendezvousReaderCount": "1",
    }       
    logging.info(" Thread %s: creating Adiosobject",name)
    adios_r = AdiosWands(link="Reader",parameters=paramsrec)
    logging.info(" Thread %s: initiating receiving",name)
    data_r = adios_r.receive("IOS","testdata")
    logging.info(" Thread %s: finished receiving",name)
    print(f"data on recv side {data_r!s}")
    name.send(data_r)

data = RandomData([5,4])
format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO,
                    datefmt="%H:%M:%S")
master_proc, receiver_proc = Pipe()
sender_proc, master_proc2 = Pipe()
#sender_proc, receiver_proc = None, None
s = Process(target=thread_send, args=[sender_proc])
r = Process(target=thread_receive,args=[receiver_proc])
s.start()
r.start()
master_proc2.send(data.get_numpy_array())
data_r = master_proc.recv()
#data_r = None
print(f"data in master {data_r!s}")
r.join()
s.join()
assert np.array_equal(data.get_numpy_array(), data_r)


In [ ]:

from wands import AdiosWands
import numpy as np
from multiprocessing import Process, Pipe
import logging
import time
import concurrent.futures

def thread_send(name:str):
   # assert name == "sender_proc"
    paramssend = {
        "IPAddress": "127.0.0.1",
        "Port": "12307",
        "Timeout": "6",
        "TransportMode": "reliable",
        "RendezvousReaderCount": "1",
    }
    data = name.recv()
    #data = np.arange(20).reshape(4,5)
    print(f"data on sender side \n{data!s}")
    #data = np.arange(20).reshape(4,5)
    logging.info(f"Sender: creating Adiosobject")
    adios_s = AdiosWands(link="Sender",parameters=paramssend)
    logging.info(f"Sender: initiating sending")
    adios_s.send("IOS", "testdata", data)
    logging.info(f"Sender: sending finished")

def thread_receive(name:str):
   # assert name == "receiver_proc"
    paramsrec =  {
        "IPAddress": "127.0.0.1",
        "Port": "12307",
        "Timeout": "6",
        "TransportMode": "reliable",
        "RendezvousReaderCount": "1",
    }       
    logging.info(f" Receiver: creating Adiosobject ")
    adios_r = AdiosWands(link="Reader",parameters=paramsrec)
    logging.info(f" Receiver: initiating receiving ")
    data_r = adios_r.receive("IOS","testdata")
    logging.info(f" Receiver: finished receiving",)
    print(f"data on recv side \n{data_r!s}")
    name.send(data_r)

#data = np.random.rand(20,1) #no exact 0 or 1 depends about the value I set in adios.py to preallocate the array
#data = np.random.rand(1,20).reshape(4,5) #workds
#data = np.arange(1,20) # no random almost 0
#data = np.arange(1,21).reshape(4,5) +1
#data = np.full([4,5],7) # no random almost 0
#data = np.ones([20,1,1])*7 #works
data = np.random.rand(4,5) #works
format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO,
                    datefmt="%H:%M:%S")
master_proc, receiver_proc = Pipe()
sender_proc, master_proc2 = Pipe()
#sender_proc, receiver_proc = None, None
s = Process(target=thread_send, args=[sender_proc])
r = Process(target=thread_receive,args=[receiver_proc])
s.start()
r.start()
master_proc2.send(data)
data_r = master_proc.recv()
#data_r = None
print(f"data in master \n{data_r!s}")
r.join()
s.join()
assert np.array_equal(data, data_r)


In [ ]:

import numpy as np
from multiprocessing import Process, Pipe
import logging
import adios2

def thread_send(name:str):

    data = name.recv() #receive data from main thread
    shape = data.shape
    count = shape
    start = (0,)*len(shape)
    print(f"data on sender side \n{data!s}")

    adios_io = adios2.ADIOS()
    wan = adios_io.DeclareIO("Server")
    wan.SetEngine("Dataman")

    wan.SetParameters(
        {
            "IPAddress": "0.0.0.0",
            "Port": "12306",
            "Timeout": "5",
            "TransportMode": "reliable",
            "RendezvousReaderCount":"1",
        }
    )
    logging.info(f"Sender: initiating sending")
    writer = wan.Open("testdata_sender", adios2.Mode.Write)
    sendbuffer = wan.DefineVariable("np_data",data, shape, start, count, adios2.ConstantDims)
    if sendbuffer:
        writer.BeginStep()
        writer.Put(sendbuffer,data,adios2.Mode.Deferred)
        writer.EndStep()
    else:
        raise ValueError("DefineVariable failed")
    
    writer.Close()
  
    logging.info(f"Sender: sending finished")

def thread_receive(name:str):

    adios_io = adios2.ADIOS()
    wan = adios_io.DeclareIO("Client")
    wan.SetEngine("Dataman")
    wan.SetParameters(
        {
            "IPAddress": "0.0.0.0",
            "Port": "12306",
            "Timeout": "5",
            "TransportMode": "reliable",
            "RendezvousReaderCount":"1",
        }
    )   
    logging.info(f" Receiver: initiating receiving ")
    reader = wan.Open("testdata_receiver", adios2.Mode.Read)
    while True:
        stepStatus = reader.BeginStep()
        if stepStatus == adios2.StepStatus.OK:
            #inquire for variable
            recvar = wan.InquireVariable("np_data")
            if recvar:
                # determine the shape of the data that will be sent
                bufshape = recvar.Shape()
                # allocate buffer for now numpy
                data = np.ones(bufshape)
                #print(f"data before Get: \n{data!s}")
                reader.Get(recvar,data,adios2.Mode.Deferred)
                #print(f"data right after get This might be not right as data might not have been sent yet \n: {data!s}")
            else:
                raise ValueError(f"InquireVariable failed")
        elif stepStatus == adios2.StepStatus.EndOfStream:
            break
        else: 
            raise StopIteration(f"next step failed to initiate {stepStatus!s}")
        reader.EndStep()
        #print(f"After end step \n{data!s}")
    reader.Close()
    #print(f"after close \n {data!s}")
    logging.info(f" Receiver: finished receiving",)
    name.send(data)

"""
Different test data arrays. 
"""
#data = np.random.rand(1,20) #no but doesn't even override the initialized data array therefore result is all 1. Does not receive data at all?
#data = np.arange(1,20) # receives random 0s
#data = np.arange(1,21).reshape(4,5) # compared to the random array reshape does not help here still receives random 0s
#data = np.full([4,5],7) # receives random 0s

#data = np.random.rand(1,20).reshape(4,5) #works
#data = np.ones([20,1])*7 #works
#data = np.ones([4,5])
data = np.random.rand(4,5) #works
print(isinstance(data,np.ndarray))
format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO,
                    datefmt="%H:%M:%S")
master_proc, receiver_proc = Pipe()
sender_proc, master_proc2 = Pipe()
s = Process(target=thread_send, args=[sender_proc])
r = Process(target=thread_receive,args=[receiver_proc])
s.start()
r.start()
master_proc2.send(data)
data_r = master_proc.recv()
#data_r = None
print(f"data in master \n{data_r!s}")
r.join()
s.join()
assert np.array_equal(data, data_r)


In [ ]:
import numpy as np

matrix = np.arange(0,30).reshape(5,6)
print(matrix)

idx = ((2,2,1),(2,2,2))
print(len(idx[0]))
print(matrix[idx])


In [ ]:
from wands import HDF5, AdiosWands
import numpy as np


hdf5file = "./data/30420.h5"
axis = "/xmc/XMC/ACQ196_143/CH01"
h5obj = HDF5(hdf5file)
rdo = h5obj.getaxis(axis)
print(rdo)

errors = rdo.get_errors()
if np.any(errors):
    print(f"an nonzero element found")
else:
    print(f"all elements are zero")


times = rdo.get_time()
print(times)


In [1]:
sendmatrix = []
sendmatrix2 = []
array1 = [1,3,3,3,3,3,3,5,3,4]
array2 = [2,5,5,3,2,4,5,6,4,6]
print(array1)
print(array2)
print(sendmatrix)

sendmatrix = [sendmatrix, array1]
print(sendmatrix)
sendmatrix = [sendmatrix,array2]
print(sendmatrix)

sendmatrix2.append(array1)
print(sendmatrix2)
sendmatrix2.append(array2)
print(sendmatrix2)

sendmatrix_np = np.array(sendmatrix2)
sendmatrix_np.shape
print(sendmatrix_np)
print(sendmatrix_np[0,:])
if sendmatrix_np.shape[0] == 2:
    print("test")

[1, 3, 3, 3, 3, 3, 3, 5, 3, 4]
[2, 5, 5, 3, 2, 4, 5, 6, 4, 6]
[]
[[], [1, 3, 3, 3, 3, 3, 3, 5, 3, 4]]
[[[], [1, 3, 3, 3, 3, 3, 3, 5, 3, 4]], [2, 5, 5, 3, 2, 4, 5, 6, 4, 6]]
[[1, 3, 3, 3, 3, 3, 3, 5, 3, 4]]
[[1, 3, 3, 3, 3, 3, 3, 5, 3, 4], [2, 5, 5, 3, 2, 4, 5, 6, 4, 6]]


NameError: name 'np' is not defined

In [1]:
from wands import AdiosWands, HDF5, RawData
import numpy as np
from multiprocessing import Process, Pipe
import logging
import time
import concurrent.futures

def thread_send(name:str):
   # assert name == "sender_proc"
    paramssend = {
        "IPAddress": "127.0.0.1",
        "Port": "12307",
        "Timeout": "6",
        "TransportMode": "reliable",
        "RendezvousReaderCount": "1",
    }
    data = name.recv()
    #logging.info(f"Sender: creating Adiosobject")
    adios_s = AdiosWands(link="Sender",parameters=paramssend)
    #logging.info(f"Sender: initiating sending")
    adios_s.send("IOS", "testdata", data)
    #logging.info(f"Sender: sending finished")

def thread_receive(name:str):
   # assert name == "receiver_proc"
    paramsrec =  {
        "IPAddress": "127.0.0.1",
        "Port": "12307",
        "Timeout": "6",
        "TransportMode": "reliable",
        "RendezvousReaderCount": "1",
    }       
    #logging.info(f" Receiver: creating Adiosobject ")
    adios_r = AdiosWands(link="Reader",parameters=paramsrec)
    #logging.info(f" Receiver: initiating receiving ")
    data_r = adios_r.receive("IOS","testdata")
    #logging.info(f" Receiver: finished receiving",)
    name.send(data_r)

hdf5f = "./data/30420.h5"
axis = "/xmc/XMC/ACQ196_143/CH01"
h5obj = HDF5(hdf5f)
rdo = h5obj.getaxis(axis)
print(rdo)

format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO,
                    datefmt="%H:%M:%S")
master_proc, receiver_proc = Pipe()
sender_proc, master_proc2 = Pipe()
#sender_proc, receiver_proc = None, None
s = Process(target=thread_send, args=[sender_proc])
r = Process(target=thread_receive,args=[receiver_proc])
s.start()
r.start()
master_proc2.send(rdo)
rdo_arrayform_r = master_proc.recv()

print(f"data in master \n{rdo_arrayform_r!s}")
r.join()
s.join()
rdo_r = RawData(axis)
rdo_r.convert_to_rawdata(rdo_arrayform_r)
print(rdo_r)
#assert np.array_equal(data, data_r)

/xmc/XMC/ACQ196_143/CH01: 
 data = [-0.00416018 -0.00385162 -0.00416018 ... -0.00416018 -0.00354307
 -0.00385162] 
 time = [-0.0999992 -0.0999972 -0.0999952 ...  1.1999948  1.1999968  1.1999988] 
 time attributes {'units': 's'} 
 errors = [0. 0. 0. ... 0. 0. 0.] 
 group attributes = {'description': '', 'label': 'Volt', 'pass_date': '2013-09-25', 'rank': 1, 'shape': array([650000], dtype=int32), 'time_index': 0, 'units': 'V'}  


22:28:56: in receive


just got into receive
after open
shape in send: (3, 650000)

22:28:56: opened
22:28:56: Attribute



count in send (3, 650000)


Process Process-2:


start in send (0, 0)

Traceback (most recent call last):


  File "/home/stefanie/software/anaconda/anaconda3/envs/adios2py/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/stefanie/software/anaconda/anaconda3/envs/adios2py/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_1223281/3652548861.py", line 36, in thread_receive
    data_r = adios_r.receive("IOS","testdata")
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/stefanie/work/adios/adiosnetwork/src/wands/adios.py", line 310, in receive
    logging.info(self._io.InquireAttribute(variable_name+'/name').Data())
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ValueError: [Tue Feb 28 22:28:56 2023] [ADIOS2 EXCEPTION] <Helper> <adiosType> <CheckForNullPtr> : found null pointer in call to Attribute::Data



In [4]:
teststr = 'abc'

teststrarray = ['abd','123','test']
print(type(teststr))
print(type(teststrarray))
test2 = [teststr]
teststr = [teststr]
for k in teststr:
    print(k)

for i in teststrarray:
    print(i)

for k in test2:
    print(k)

<class 'str'>
<class 'list'>
abc
abd
123
test
abc
